In [1]:
import ctypes

In [2]:
dll_path = 'C:/Program Files/Thorlabs/MCM3000 4.0/Application/ThorMCM3000.dll'

In [3]:
device = ctypes.windll.LoadLibrary(dll_path)
# functions return 0 is failed, 1 if succeded
#dev = ctypes.windll.ThorMCM3000

In [10]:
n_devices = ctypes.c_long()
i_device = ctypes.c_long(0)

In [5]:
#device.FindDevices.argtypes = [ctypes.c_long]
#device.FindDevices.restype = ctypes.c_bool

In [12]:
device.FindDevices(ctypes.byref(n_devices))

0

In [11]:
device.SelectDevice(i_device)

0

In [7]:
n_devices

c_long(0)

In [18]:
error_msg = ctypes.create_string_buffer(32)

In [20]:
device.GetLastErrorMsg(ctypes.byref(error_msg), 32)

1

In [21]:
error_msg.value

b''

In [35]:
# parameter IDs
PARAM_DEVICE_TYPE = 0
PARAM_X_POS = 200
PARAM_X_HOME = 201
PARAM_X_ZERO = 202
PARAM_X_POS_CURRENT = 207
PARAM_X_STOP = 209
PARAM_X_VELOCITY_CURRENT = 210
PARAM_Y_POS = 300
PARAM_Y_HOME = 301
PARAM_Y_ZERO = 302
PARAM_Y_POS_CURRENT = 307
PARAM_Y_STOP = 309
PARAM_Y_VELOCITY_CURRENT = 310
PARAM_Z_POS = 400
PARAM_Z_HOME = 401
PARAM_Z_ZERO = 402
PARAM_Z_POS_CURRENT = 407
PARAM_Z_STOP = 409
PARAM_Z_VELOCITY_CURRENT = 410

STATUS_BUSY = 0
STATUS_READY = 1

In [32]:
paramType = ctypes.c_int32()
paramAvailable = ctypes.c_int32()
paramReadOnly = ctypes.c_int32()
paramMin = ctypes.c_double()
paramMax = ctypes.c_double()
paramDefault = ctypes.c_double()
status = ctypes.c_int32()

In [16]:
device.GetParamInfo(PARAM_DEVICE_TYPE, ctypes.byref(paramType), 
                    ctypes.byref(paramAvailable), 
                    ctypes.byref(paramReadOnly), 
                    ctypes.byref(paramMin), 
                    ctypes.byref(paramMax), 
                    ctypes.byref(paramDefault))

0

In [11]:
device.GetParamInfo(PARAM_X_POS, ctypes.byref(paramType), 
                    ctypes.byref(paramAvailable), 
                    ctypes.byref(paramReadOnly), 
                    ctypes.byref(paramMin), 
                    ctypes.byref(paramMax), 
                    ctypes.byref(paramDefault))

0

In [26]:
posx = ctypes.c_double()
posx.value = paramMax.value/10.
print(posx)

c_double(10.0)


In [30]:
device.SetParam(PARAM_X_POS, posx)
device.PreflightPosition()
device.SetupPosition()
device.StartPosition()

0

In [36]:
device.StatusPosition(ctypes.byref(status))

1

In [37]:
status

c_long(1)

In [ ]:
# add while loop to poll status

In [38]:
device.PostflightPosition()

1

In [44]:
device.GetParam(PARAM_X_POS_CURRENT, posx)

0

In [45]:
posx

c_double(10.0)

In [1]:
import serial

In [3]:
serial.Serial?

In [283]:
ser = serial.Serial('COM32', 460800, timeout=2)

In [284]:
print(ser.isOpen())

True


In [275]:
def get_current_position(ser):
    ser.flushInput()
    ser.flushOutput()
    command = b'\x0A\x04\x00\x00\x00\x00'
    ser.write(command)
    response = ser.read(size=12)
    encoder_count = int.from_bytes(response[-4:], 'little')
    return encoder_count

In [285]:
get_current_position(ser)

4294957849

In [57]:
def set_zero(ser):
    ser.flushInput()
    ser.flushOutput()
    command = b'\x09\x04\x06\x00\x00\x00\x00\x00\x00\x00\x00\x00'
    ser.write(command)

In [286]:
set_zero(ser)

In [234]:
# stage ZFM2020
def um2counts(micron):
    return int(micron / 0.2116667)

def counts2mm(counts):
    return counts * 0.2116667

In [287]:
import struct
def go_to_position(ser, posx_um=0):
    ser.flushInput()
    ser.flushOutput()
    counts_binary = struct.pack("<l", um2counts(posx_um))
    command = b'\x53\x04\x06\x00\x00\x00\x00\x00' + counts_binary
    ser.write(command)

In [295]:
import time
go_to_position(ser,5000)
time.sleep(0.4)
stop(ser)

In [289]:
def stop(ser):
    ser.flushInput()
    ser.flushOutput()
    command = b'\x65\x04\x00\x01\x00\x00'
    ser.write(command)

In [280]:
ser.close()